Modify the exisitng models reliably predict if new
s is fake or real, uisng exisitng KAGGEL datasets

#### Dataset used - https://www.kaggle.com/fake-news/data

### Dataset Description

train.csv: A full training dataset with the following attributes:

* id: unique id for a news article
* title: the title of a news article
* author: author of the news article
* text: the text of the article; could be incomplete
* label: a label that marks the article as potentially unreliable
  * 1: FAKE
  * 0: TRUE


Set the dependencies

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import re 
import nltk 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics
import itertools


The Natural Language Toolkit (NLTK) is a platform used for building Python programs that work with human language data for applying in statistical natural language processing (NLP). It contains text processing libraries for tokenization, parsing, classification, stemming, tagging and semantic reasoning.
The nltk.corpus package defines a collection of corpus reader classes, which can be used to access the contents of a diverse set of corpora. The list of available corpora is given at: https://www.nltk.org/nltk_data/ Each corpus reader class is specialized to handle a specific corpus forma

Stopwords are words which occur frequently in a corpus. e.g a, an, the, in. Frequently occurring words are removed from the corpus for the purpose of text-normalization.

In [93]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [94]:
# check if Stopwords laoded in english

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing and Analysis


Regular Expression Syntax. A regular expression (or RE) specifies a set of strings that matches it; the functions in this module let you check if a particular string matches a given regular expression (or if a given regular expression matches a particular string, which comes down to the same thing

Load the dataset, Read

In [96]:
# First let's load the dataset
news_dataset = pd.read_csv('trainsmall.csv',encoding = "latin-1")


<ipython-input-96-3689a30b2268>:2: DtypeWarning: Columns (73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249) have mixed types. Specify dtype option on import or set low_memory=False.
  news_dataset = pd.read_csv('trainsmall.csv',encoding = "latin-1")


In [97]:
news_dataset.head()

,id,title,author,text,label,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 241,Unnamed: 242,Unnamed: 243,Unnamed: 244,Unnamed: 245,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
news_dataset.shape

(5867, 251)

In [99]:
# Checking for missing values
news_dataset.isnull().sum()

id                 0
title            460
author           989
text             619
label            713
                ... 
Unnamed: 246    5866
Unnamed: 247    5866
Unnamed: 248    5866
Unnamed: 249    5866
Unnamed: 250    5866
Length: 251, dtype: int64

In [100]:
# Let's replace the missing values with empty strings wherever possible
news_dataset = news_dataset.fillna('')

In [101]:
news_dataset.isnull().sum()

id              0
title           0
author          0
text            0
label           0
               ..
Unnamed: 246    0
Unnamed: 247    0
Unnamed: 248    0
Unnamed: 249    0
Unnamed: 250    0
Length: 251, dtype: int64

In thsi dataset the Title, Author indicates validity of the message

In [102]:
news_dataset['text_corpus'] = news_dataset['author']+' '+news_dataset['title']+' '+news_dataset['text']

<ipython-input-102-262e9a411340>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  news_dataset['text_corpus'] = news_dataset['author']+' '+news_dataset['title']+' '+news_dataset['text']


In [103]:
print(news_dataset['text_corpus'])

0       Darrell Lucus House Dem Aide: We Didnât Even...
1       Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2       Consortiumnews.com Why the Truth Might Get You...
3       Jessica Purkiss 15 Civilians Killed In Single ...
4       Howard Portnoy Iranian woman jailed for fictio...
                              ...                        
5862    Samuel Chi California Ready for Single-Payer I...
5863    Mac Slavo âViolent Revolution If Trump Lets ...
5864    Amanda Hess How Conservative Sites Turn Celebr...
5865    David Gutierrez, staff writer Woman awarded $7...
5866    Simon Kent Artist Says Burqa Images Challenge ...
Name: text_corpus, Length: 5867, dtype: object


In [104]:
news_dataset.head()

,id,title,author,text,label,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 242,Unnamed: 243,Unnamed: 244,Unnamed: 245,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,text_corpus
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1,,,,,,...,,,,,,,,,,Darrell Lucus House Dem Aide: We Didnât Even...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,,,,,,...,,,,,,,,,,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,,,,,,...,,,,,,,,,,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,,,,,,...,,,,,,,,,,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1,,,,,,...,,,,,,,,,,Howard Portnoy Iranian woman jailed for fictio...


In [105]:
# Remove all columns between column index 1 to 3
news_dataset.drop(news_dataset.iloc[:, 5:250], inplace=True, axis=1)
news_dataset.head()

,id,title,author,text,label,Unnamed: 250,text_corpus
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1,,Darrell Lucus House Dem Aide: We Didnât Even...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1,,Howard Portnoy Iranian woman jailed for fictio...


In [106]:
# Now we will separate the data and label i.e. text_corpus and label fields
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [108]:
X = news_dataset['text_corpus']
print(X)

0       Darrell Lucus House Dem Aide: We Didnât Even...
1       Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2       Consortiumnews.com Why the Truth Might Get You...
3       Jessica Purkiss 15 Civilians Killed In Single ...
4       Howard Portnoy Iranian woman jailed for fictio...
                              ...                        
5862    Samuel Chi California Ready for Single-Payer I...
5863    Mac Slavo âViolent Revolution If Trump Lets ...
5864    Amanda Hess How Conservative Sites Turn Celebr...
5865    David Gutierrez, staff writer Woman awarded $7...
5866    Simon Kent Artist Says Burqa Images Challenge ...
Name: text_corpus, Length: 5867, dtype: object


In [109]:
print(Y)

0       1
1       0
2       1
3       1
4       1
       ..
5862    0
5863    1
5864    0
5865    1
5866    0
Name: label, Length: 5867, dtype: object


In [110]:
X.head()

0    Darrell Lucus House Dem Aide: We Didnât Even...
1    Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2    Consortiumnews.com Why the Truth Might Get You...
3    Jessica Purkiss 15 Civilians Killed In Single ...
4    Howard Portnoy Iranian woman jailed for fictio...
Name: text_corpus, dtype: object




Stemming or Lemmatization-https://pythonprogramming.net/stemming-nltk-tutorial/

Stemming is a technique used to reduce an inflected word down to its word stem. For example, the words “programming,” “programmer,” and “programs” can all be reduced down to the common word stem “program.” In other words, “program” can be used as a synonym for the prior three inflection words

Porter’s Stemmer applies a set of five sequential rules (also called phases) to determine common suffixes from sentences.


In [111]:
port_stem = PorterStemmer()

def stemming(content):
    # Pick all alphabet characters - lowercase and uppercase...all others such as numbers and punctuations will be removed. Numbers or punctuations will be replaced by a whitespace
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    # Converting all letters to lowercase 
    stemmed_content = stemmed_content.lower()
    # Converting all to a splitted case or a list
    stemmed_content = stemmed_content.split()
    # Applying stemming, so we get the root words wherever possible + remove stopwords as well
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    # Join all the words in final content
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [112]:
news_dataset['text_corpus'] = news_dataset['text_corpus'].apply(stemming)

In [113]:
print(news_dataset['text_corpus'])

0       darrel lucu hous dem aid even see comey letter...
1       daniel j flynn flynn hillari clinton big woman...
2       consortiumnew com truth might get fire truth m...
3       jessica purkiss civilian kill singl us airstri...
4       howard portnoy iranian woman jail fiction unpu...
                              ...                        
5862    samuel chi california readi singl payer insur ...
5863    mac slavo violent revolut trump let peopl rema...
5864    amanda hess conserv site turn celebr despair h...
5865    david gutierrez staff writer woman award contr...
5866    simon kent artist say burqa imag challeng anti...
Name: text_corpus, Length: 5867, dtype: object


In [114]:
# Separating data and label
X = news_dataset['text_corpus'].values
Y = news_dataset['label'].values

In [115]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet hous dem aid even see comey letter jason chaffetz tweet darrel lucu octob subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ common licens apolog keith olbermann doubt worst person world week fbi director jame comey accord hous democrat aid look like also know second worst person well turn comey sent infam letter announc fbi look email may relat hillari clinton email server rank democrat relev committe hear comey found via tweet one republican committe chairmen know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen jason chaffetz jasoninthehous octob cours know case comey actual say review email light unrel 

In [116]:
print(Y)

['1' '0' '1' ... '0' '1' '0']


TF-IDF (Term Frequency, Inverse Document Frequency)

### Converting Textual data to Numerical data

* We will use TF-IDF Vectorizer
* TF-IDF Vectorizer will convert textual data to numerical data

In [117]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [118]:
print(X)

  (0, 51287)	0.044258656517818414
  (0, 51262)	0.020131991582391793
  (0, 50849)	0.04475430852207461
  (0, 50847)	0.09384588101602599
  (0, 50822)	0.03825205199308433
  (0, 50797)	0.01165954825124317
  (0, 50480)	0.01738811022569182
  (0, 50102)	0.01757032961368408
  (0, 50061)	0.012836637537017756
  (0, 50015)	0.028302045379509727
  (0, 49978)	0.013052060041706186
  (0, 49880)	0.012112781491550742
  (0, 49495)	0.02523161607665395
  (0, 49478)	0.03123554543446421
  (0, 49031)	0.021201090024379893
  (0, 48472)	0.06517493117493858
  (0, 48087)	0.03247173661266174
  (0, 47937)	0.016824891525565502
  (0, 47477)	0.03647786256425233
  (0, 47221)	0.011529047763407995
  (0, 47218)	0.044338350940318454
  (0, 47183)	0.13661884669740237
  (0, 47110)	0.07276928934038554
  (0, 46494)	0.03671673505758655
  (0, 46236)	0.027634918285441675
  :	:
  (5866, 8490)	0.11549882146254178
  (5866, 8482)	0.0232165679566044
  (5866, 8325)	0.04297114826724454
  (5866, 8218)	0.03263883279035373
  (5866, 7537)	0.02

Modeling & Model Evaluation

### Splitting the data into test and train datasets

In [119]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.18, random_state=42)

### Training the model (model1 : Logistic Regression)

In [120]:
# Training the model
model1 = LogisticRegression()

model1.fit(X_train, Y_train)

LogisticRegression()

### Model Evaluation

In [121]:
# Accuracy Score on Training Data
X_train_prediction = model1.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

print('Accuracy score on the training data: ',training_data_accuracy)

# Accuracy Score on Test Data
X_test_prediction = model1.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

print('Accuracy score on the test data: ',test_data_accuracy)

Accuracy score on the training data:  0.9444906444906445
Accuracy score on the test data:  0.8789025543992431


In [122]:
# Classification report for test data
classification_report(Y_test, X_test_prediction)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          precision    recall  f1-score   support\n\n                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

**CLASSIFICATION MODEL : PASSIVE AGGRESSIVE CLASSIFIER**

* Passive Aggressive Classifier works by responding as passive for correct classifications and responding as aggressive for any miscalculation.

In [123]:
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [124]:
model2 = PassiveAggressiveClassifier()
model2.fit(X2_train,Y2_train)

PassiveAggressiveClassifier()

In [125]:
X2_train_prediction = model2.predict(X2_train)
training_data_accuracy2 = accuracy_score(X2_train_prediction, Y2_train)

print('Accuracy score on the training data: ',training_data_accuracy2)

# Accuracy Score on Test Data
X2_test_prediction = model2.predict(X2_test)
test_data_accuracy1 = accuracy_score(X2_test_prediction, Y2_test)

print('Accuracy score on the test data: ',test_data_accuracy1)

Accuracy score on the training data:  0.9427480916030534
Accuracy score on the test data:  0.835828600929272


Testing the two models

In [134]:
X_new = X_test[100]

prediction = model1.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('The news in Real')
else:
  print('The news is Fake and Unreliable')

['1']
The news is Fake and Unreliable


In [135]:
X2_new = X2_test[100]

prediction2 = model2.predict(X2_new)
print(prediction2)

if (prediction2[0] == 0):
  print('The news in Real')
else:
  print('The news is Fake and Unreliable')

['1']
The news is Fake and Unreliable


In [127]:
news_dataset[100:101]

,id,title,author,text,label,Unnamed: 250,text_corpus
100,92,"Venezuela Muzzles Legislature, Moving Closer t...",Nicholas Casey and Patricia Torres,"IQUITOS, Peru â Venezuela took its strong...",0,,nichola casey patricia torr venezuela muzzl le...


In [128]:
print(Y_test[100])

1


In [129]:
X_new = X_test[301]

prediction = model1.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('The news in Real')
else:
  print('The news is Fake and Unreliable')

['0']
The news is Fake and Unreliable


In [130]:
print(Y_test[301])

0


In [131]:
news_dataset[300:301]

,id,title,author,text,label,Unnamed: 250,text_corpus
300,291,Confused by Chip Credit Cards? Get in Line - T...,Karen Workman,Youâre at the checkout counter. Your items h...,0,,karen workman confus chip credit card get line...
